<a href="https://colab.research.google.com/github/RhiturajD/Traditional-Dance-Classifier-with-the-help-ofCNN/blob/main/Traditional_Dance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
train_df = pd.read_csv('Traditional-Dance-Classification/dataset/train.csv')
test_df = pd.read_csv('Traditional-Dance-Classification/dataset/test.csv')


In [1]:
print(train_df.head())

print(test_df.head())

NameError: ignored

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
img = cv2.imread('Traditional-Dance-Classification/dataset/train/' + str(train_df['Image'][4]))
cv2_imshow(img)
img.shape


In [ ]:
train_df['target'].value_counts()

In [1]:

from sklearn.model_selection import train_test_split
train, validate = train_test_split(train_df, test_size = 0.25, random_state = 1)

train['target'].value_counts()

validate['target'].value_counts()

'# Spliting Train and Test Data'

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import activations
from tensorflow.keras import layers
import numpy as np

In [ ]:
classifier = Sequential()

classifier.add(Conv2D(32, (3,3), input_shape = (128,128,3), activation = 'relu'))
#classifier.add(BatchNormalization())
#classifier.add(layers.Activation('relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
#classifier.add(Dropout(0.5))

classifier.add(Conv2D(64, (3,3), activation = 'relu'))
#classifier.add(BatchNormalization())
#classifier.add(layers.Activation('relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.5))

classifier.add(Conv2D(128, (3,3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.5))

classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 8, activation = 'softmax'))

classifier.compile(loss = 'categorical_crossentropy', 
                   optimizer='adam',
                   metrics = ['accuracy'])

classifier.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_image_data = ImageDataGenerator(rescale = 1./255,
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True,
                                      vertical_flip = True)
val_image_data = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_image_data.flow_from_dataframe(dataframe = train,
                                                       directory = 'Traditional-Dance-Classification/dataset/train',
                                                       x_col = 'Image',
                                                       y_col = 'target',
                                                       batch_size = 32,
                                                       class_mode = 'categorical',
                                                       shuffle = True,
                                                       target_size = (128,128))


In [ ]:
test_generator = val_image_data.flow_from_dataframe(dataframe = validate,
                                                    directory = 'Traditional-Dance-Classification/dataset/train',
                                                    x_col = 'Image',
                                                    y_col = 'target',
                                                    batch_size = 32,
                                                    class_mode = 'categorical',
                                                    shuffle = True,
                                                    target_size = (128,128))


In [ ]:
trained_model = classifier.fit(train_generator,
                               validation_data = test_generator,
                               epochs = 100)


In [ ]:
classifier.evaluate_generator(generator=test_generator)
classifier.save('trained_model.h5')


In [ ]:
import matplotlib.pyplot as plt
print(trained_model.history.keys())

plt.plot(trained_model.history['loss'])
plt.plot(trained_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='upper left')
plt.show()
